# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [105]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [106]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.drop([467,484], inplace = True )
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [107]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 87886208, 'calculationPrice': 'close', 'change': 2.2, 'changePercent': 0.01354, 'close': 0, 'closeSource': 'iacflfoi', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 165.98, 'iexCloseTime': 1683063186365, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 171.03, 'iexOpenTime': 1687945135217, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1726085392492, 'latestPrice': 165.88, 'latestSource': 'Close', 'latestTime': 'February 25, 2022', 'latestUpdate': 1687240019079, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2722053648002, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTim

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [108]:
price = data['latestPrice']
pe_ratio = data['peRatio']


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [109]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [110]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,136.09,34.39,N/A
1,AAL,17.69,-5.85,N/A
2,AAP,214.18,21.84,N/A
3,AAPL,169.39,27.39,N/A
4,ABBV,156.24,24.34,N/A
...,...,...,...,...
498,YUM,129.70,24.4,N/A
499,ZBH,127.70,32.8,N/A
500,ZBRA,437.70,26.9,N/A
501,ZION,72.33,6.9,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [111]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe.reset_index(inplace=True, drop=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
23,EBAY,57.20,2.73,N/A
24,F,18.06,4.13,N/A
25,NRG,39.55,4.37,N/A
26,BIO,651.41,4.63,N/A
27,MPC,79.89,5.23,N/A
28,PRU,113.40,5.79,N/A
29,AIG,65.07,5.86,N/A
30,COF,156.29,5.86,N/A
31,SYF,44.73,5.95,N/A
32,GS,366.28,5.96,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [112]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")
    flag=False
    while flag==False:
        try:
            portfolio_size = float(portfolio_size)
            flag=True
        except ValueError:
            print("That's not a number! \n Try again:")
            portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [113]:
portfolio_input()

Enter the value of your portfolio:2500000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [114]:
position_size = float(portfolio_size)/len(final_dataframe)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
23,EBAY,57.20,2.73,874
24,F,18.06,4.13,2768
25,NRG,39.55,4.37,1264
26,BIO,651.41,4.63,76
27,MPC,79.89,5.23,625
28,PRU,113.40,5.79,440
29,AIG,65.07,5.86,768
30,COF,156.29,5.86,319
31,SYF,44.73,5.95,1117
32,GS,366.28,5.96,136


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [115]:
symbol = 'AAPL'
batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url).json()
print(data[symbol]['advanced-stats'])
#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
    
#Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gp = enterprise_value/gross_profit


{'beta': 1.331099386218573, 'totalCash': 66747081202, 'currentDebt': 154227318101, 'revenue': 393634120743, 'grossProfit': 168084355966, 'totalRevenue': 396672424989, 'EBITDA': 132074402483, 'revenuePerShare': 24.31, 'revenuePerEmployee': 2601676.71, 'debtToEquity': 5.448284131998145, 'profitMargin': 0.2744693472678859, 'enterpriseValue': 2779798297073, 'enterpriseValueToRevenue': 7.61, 'priceToSales': 7.21, 'priceToBook': 38.8, 'forwardPERatio': 28.106914778152117, 'pegRatio': 0.4402604860651433, 'peHigh': 27.876780186312203, 'peLow': 14.594343767430859, 'week52highDate': '2021-12-27', 'week52lowDate': '2021-03-04', 'putCallRatio': 0.5162638635988146, 'companyName': 'Apple Inc', 'marketcap': 2726246529308, 'week52high': 187.17, 'week52low': 120.56, 'week52highSplitAdjustOnly': 184.42, 'week52highDateSplitAdjustOnly': '2022-01-03', 'week52lowSplitAdjustOnly': 118.37, 'week52lowDateSplitAdjustOnly': '2021-03-02', 'week52change': 0.3690316620604303, 'sharesOutstanding': 17041799260, 'flo

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [116]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
    
]
rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ], index = rv_columns
            ), ignore_index = True
        )

In [117]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.83,N/A,34.63,N/A,7.57,N/A,6.37,N/A,24.609237,N/A,12.181770,N/A,N/A
1,AAL,17.51,N/A,-5.93,N/A,-1.59,N/A,0.3823,N/A,-12.498201,N/A,1.276123,N/A,N/A
2,AAP,212.48,N/A,22.1,N/A,4.06,N/A,1.18,N/A,11.811585,N/A,2.532867,N/A,N/A
3,AAPL,172.71,N/A,27.83,N/A,38.7,N/A,7.32,N/A,21.432185,N/A,17.231161,N/A,N/A
4,ABBV,150.83,N/A,24.26,N/A,17.74,N/A,4.83,N/A,11.520822,N/A,8.226490,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,127.10,N/A,24.9,N/A,-4.45,N/A,5.68,N/A,20.530644,N/A,9.688879,N/A,N/A
499,ZBH,130.50,N/A,32.6,N/A,2.18,N/A,3.43,N/A,18.850012,N/A,6.095880,N/A,N/A
500,ZBRA,430.03,N/A,26.7,N/A,7.56,N/A,4.07,N/A,20.342380,N/A,8.755644,N/A,N/A
501,ZION,73.69,N/A,7.01,N/A,1.55,N/A,3.31,N/A,5.321296,N/A,2.888600,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [118]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,186.600,N/A,60.49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,307.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,329.090,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.89,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,28.690,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.990,N/A,14.66,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.500,N/A,32.75,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.830,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,123.350,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2810.360,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [119]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [120]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [121]:
from scipy.stats import percentileofscore as score
metrics = {'Price-to-Earnings Ratio':'PE Percentile',
'Price-to-Book Ratio':'PB Percentile',
'Price-to-sales Ratio':'PS Percentile',
'EV/EBITDA':'EV/EBITDA Percentile',
'EV/GP':'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
rv_dataframe
    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.83,N/A,34.63,73.3598,7.57,78.1312,6.3700,80.5169,24.609237,81.7097,12.181770,80.3181,N/A
1,AAL,17.51,N/A,-5.93,3.57853,-1.59,5.76541,0.3823,2.58449,-12.498201,4.77137,1.276123,4.37376,N/A
2,AAP,212.48,N/A,22.10,46.1233,4.06,59.0457,1.1800,15.1093,11.811585,38.5686,2.532867,12.1272,N/A
3,AAPL,172.71,N/A,27.83,60.6362,38.70,96.8191,7.3200,83.6978,21.432185,77.1372,17.231161,91.6501,N/A
4,ABBV,150.83,N/A,24.26,51.0934,17.74,92.6441,4.8300,70.0795,11.520822,37.7734,8.226490,55.666,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,127.10,N/A,24.90,52.8827,-4.45,5.36779,5.6800,76.7396,20.530644,72.7634,9.688879,69.3837,N/A
499,ZBH,130.50,N/A,32.60,70.7753,2.18,30.5169,3.4300,51.6899,18.850012,67.7932,6.095880,40.7555,N/A
500,ZBRA,430.03,N/A,26.70,59.0457,7.56,77.833,4.0700,59.6421,20.342380,71.9682,8.755644,60.0398,N/A
501,ZION,73.69,N/A,7.01,8.94632,1.55,17.7932,3.3100,49.7018,5.321296,10.5368,2.888600,14.5129,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [122]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles=[]
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.83,N/A,34.63,73.3598,7.57,78.1312,6.3700,80.5169,24.609237,81.7097,12.181770,80.3181,78.8072
1,AAL,17.51,N/A,-5.93,3.57853,-1.59,5.76541,0.3823,2.58449,-12.498201,4.77137,1.276123,4.37376,4.21471
2,AAP,212.48,N/A,22.10,46.1233,4.06,59.0457,1.1800,15.1093,11.811585,38.5686,2.532867,12.1272,34.1948
3,AAPL,172.71,N/A,27.83,60.6362,38.70,96.8191,7.3200,83.6978,21.432185,77.1372,17.231161,91.6501,81.9881
4,ABBV,150.83,N/A,24.26,51.0934,17.74,92.6441,4.8300,70.0795,11.520822,37.7734,8.226490,55.666,61.4513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,127.10,N/A,24.90,52.8827,-4.45,5.36779,5.6800,76.7396,20.530644,72.7634,9.688879,69.3837,55.4274
499,ZBH,130.50,N/A,32.60,70.7753,2.18,30.5169,3.4300,51.6899,18.850012,67.7932,6.095880,40.7555,52.3062
500,ZBRA,430.03,N/A,26.70,59.0457,7.56,77.833,4.0700,59.6421,20.342380,71.9682,8.755644,60.0398,65.7058
501,ZION,73.69,N/A,7.01,8.94632,1.55,17.7932,3.3100,49.7018,5.321296,10.5368,2.888600,14.5129,20.2982


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [123]:
rv_dataframe.sort_values('RV Score', ascending= True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,17.51,N/A,-5.93,3.57853,-1.5900,5.76541,0.3823,2.58449,-12.498201,4.77137,1.276123,4.37376,4.21471
1,UNM,28.78,N/A,7.54,10.338,0.5261,5.96421,0.4450,3.37972,3.538785,6.36183,0.441590,0.596421,5.32803
2,PRU,113.70,N/A,6.04,6.16302,0.7069,6.75944,0.6191,6.16302,4.014596,7.55467,0.595114,0.994036,5.52684
3,AIG,63.22,N/A,6.12,6.56064,0.8120,7.15706,1.0300,12.7237,3.732205,6.56064,0.989216,2.58449,7.1173
4,GM,48.32,N/A,7.36,9.64215,1.1900,10.338,0.5533,5.36779,2.559747,5.96421,2.157308,10.1392,8.29026
5,MET,68.83,N/A,9.75,13.8171,0.8416,7.35586,0.8032,8.94632,5.195155,10.338,0.786360,1.59046,8.40954
6,LNC,69.61,N/A,9.50,12.7237,0.5937,6.16302,0.6330,6.36183,6.632217,15.9046,0.598276,1.19284,8.46918
7,F,17.95,N/A,4.07,4.97018,1.5000,15.9046,0.5354,4.77137,4.648371,8.94632,2.462608,11.9284,9.30417
8,HFC,29.97,N/A,9.10,12.5249,0.8684,7.75348,0.2745,1.19284,6.253698,14.1153,2.458197,11.7296,9.46322
9,KSS,58.94,N/A,8.98,11.9284,1.6800,20.8748,0.4190,3.18091,4.095707,7.75348,1.260627,3.77734,9.50298


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [124]:
portfolio_input()

Enter the value of your portfolio:2500000


In [125]:
position_size = portfolio_size/len(rv_dataframe)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

/home/souvik/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,17.51,2855,-5.93,3.57853,-1.5900,5.76541,0.3823,2.58449,-12.498201,4.77137,1.276123,4.37376,4.21471
1,UNM,28.78,1737,7.54,10.338,0.5261,5.96421,0.4450,3.37972,3.538785,6.36183,0.441590,0.596421,5.32803
2,PRU,113.70,439,6.04,6.16302,0.7069,6.75944,0.6191,6.16302,4.014596,7.55467,0.595114,0.994036,5.52684
3,AIG,63.22,790,6.12,6.56064,0.8120,7.15706,1.0300,12.7237,3.732205,6.56064,0.989216,2.58449,7.1173
4,GM,48.32,1034,7.36,9.64215,1.1900,10.338,0.5533,5.36779,2.559747,5.96421,2.157308,10.1392,8.29026
5,MET,68.83,726,9.75,13.8171,0.8416,7.35586,0.8032,8.94632,5.195155,10.338,0.786360,1.59046,8.40954
6,LNC,69.61,718,9.50,12.7237,0.5937,6.16302,0.6330,6.36183,6.632217,15.9046,0.598276,1.19284,8.46918
7,F,17.95,2785,4.07,4.97018,1.5000,15.9046,0.5354,4.77137,4.648371,8.94632,2.462608,11.9284,9.30417
8,HFC,29.97,1668,9.10,12.5249,0.8684,7.75348,0.2745,1.19284,6.253698,14.1153,2.458197,11.7296,9.46322
9,KSS,58.94,848,8.98,11.9284,1.6800,20.8748,0.4190,3.18091,4.095707,7.75348,1.260627,3.77734,9.50298


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [126]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [127]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [128]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [129]:
writer.save()